In [ ]:
import json

import requests
from flask import Flask, request, render_template
from wtforms import Form, TextField, validators

import config

indexer_url = config.indexer_url
text_processing_url = config.text_processing_url
ranking_url = config.ranking_url
snippets_url = config.snippets_url
res_page_form_url = config.res_page_form_url


In [ ]:
class ReusableForm(Form):
    name = TextField('Enter your request:', validators=[validators.DataRequired()])


app = Flask(__name__)


@app.route('/', methods=['GET', 'POST'])
def index():
    """
    If GET -> renders Main page from templates/index.html
    If POST -> renders ranged search results
    Parameters:
    query: str
    """
    form = ReusableForm(request.form)
    if request.method == 'POST':
        query = request.form['name']
        r = requests.post(text_processing_url + config.NORMALIZE_QUERY_PATH,
                          json=query)
        search_query = r.text
        r = requests.post(indexer_url + config.SEARCH_PATH,
                          json=search_query)
        search_results = json.loads(r.text, encoding="utf-8")
        
        r = requests.post(ranking_url + config.RANK_PATH,
                          json=search_query)
        
        search_results = json.loads(r.text, encoding="utf-8")
        
        r = requests.post(snippets_url + config.SNIPPETS_PATH,
                          json=search_results)
        snipets = json.loads(r.text, encoding="utf-8")
        
        r = requests.post(res_page_form_url + config.RESULT_PAGE_PATH,
                          json=snipets)
        result_page = r.text
        return result_page
    else:
        return render_template('index.html', form=form)


In [ ]:
if __name__ == "__main__":
    app.run(host=config.MANAGER_HOST, port=config.MANAGER_PORT)
